In [8]:
from pathlib import Path
import databento as db
import pandas as pd

base = Path(r"C:\Users\alexp\OneDrive\Gdrive\Trading\GitHub Projects\databento-es-options")
dbn_path = base / "GLBX-20251022-SPHWKWWYWA" / "glbx-mdp3-20251015-20251021.ohlcv-1m.dbn.zst"

print("Using:", dbn_path, "| Exists:", dbn_path.exists())

# Open DBN file
store = db.DBNStore.from_file(str(dbn_path))
print("Schema:", store.schema)

# ✅ Load the entire file to a DataFrame
# (for large data, this can take a few seconds)
df = store.to_df()
print("\nLoaded DataFrame shape:", df.shape)

# ✅ Display a small preview
print(df.head(20))
print("\nColumns:", list(df.columns))
print("\nData types:")
print(df.dtypes)


Using: C:\Users\alexp\OneDrive\Gdrive\Trading\GitHub Projects\databento-es-options\GLBX-20251022-SPHWKWWYWA\glbx-mdp3-20251015-20251021.ohlcv-1m.dbn.zst | Exists: True
Schema: ohlcv-1m

Loaded DataFrame shape: (15233, 9)
                           rtype  publisher_id  instrument_id    open    high  \
ts_event                                                                        
2025-10-15 00:11:00+00:00     33             1       42080918   75.25   75.25   
2025-10-15 00:11:00+00:00     33             1        4249046  117.50  117.50   
2025-10-15 00:15:00+00:00     33             1         274794   14.00   14.00   
2025-10-15 00:32:00+00:00     33             1        2570595   -0.50   -0.50   
2025-10-15 00:37:00+00:00     33             1        4249046  119.25  119.25   
2025-10-15 00:38:00+00:00     33             1        4249046  119.25  119.25   
2025-10-15 00:38:00+00:00     33             1         544348   15.75   15.75   
2025-10-15 00:39:00+00:00     33             1    

In [9]:
df = store.to_df()  # you already did this
df = df.reset_index(names='ts_event').sort_values('ts_event')
df['ts_min'] = df['ts_event'].dt.floor('T')  # minute bucket
print("Unique symbols:", df['symbol'].nunique())
print(df['symbol'].value_counts().head(10))


Unique symbols: 1740
symbol
ESZ5 P6500    339
ESZ5 C7000    312
ESZ5 C6800    312
ESZ5 C6900    296
ESZ5 P6000    257
ESZ5 P6300    251
ESZ5 P6700    231
ESZ5 P6600    212
ESZ5 P5800    193
ESZ5 P6200    162
Name: count, dtype: int64


C:\Users\alexp\AppData\Local\Temp\ipykernel_35164\3171853774.py:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df['ts_min'] = df['ts_event'].dt.floor('T')  # minute bucket


In [14]:
import pandas as pd

# --- 0) Start from your loaded df ---
# df = store.to_df()  # if you need to reload

# --- 1) Normalize ts_event -> ensure ONE tz-aware UTC column ---
if isinstance(df.index, pd.DatetimeIndex):
    # If it's the index, make sure it's tz-aware UTC
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC")
    else:
        df.index = df.index.tz_convert("UTC")
    # Move to column named ts_event
    df = df.reset_index().rename(columns={"index": "ts_event"})
else:
    # It's already a column — force UTC tz-aware
    df["ts_event"] = pd.to_datetime(df["ts_event"], utc=True)

# --- 2) Create a minute bucket (still UTC aware) ---
df = df.sort_values("ts_event")
df["ts_min"] = df["ts_event"].dt.floor("T")

# One row per symbol-minute
dedup = df.drop_duplicates(["symbol", "ts_min"])

# --- 3) Coverage per symbol (no global union; simple per-symbol range) ---
def coverage_per_symbol(g):
    start = g["ts_min"].min()
    end   = g["ts_min"].max()
    # full minute grid within that symbol’s observed span
    full = pd.date_range(start=start, end=end, freq="T", tz="UTC")
    present = pd.DatetimeIndex(g["ts_min"].unique()).tz_convert("UTC")
    missing = full.difference(present)
    return pd.Series({
        "minutes_present": len(present),
        "minutes_expected": len(full),
        "minutes_missing": len(missing),
        "coverage_pct": (len(present) / len(full)) if len(full) else float("nan"),
        "first_ts": start,
        "last_ts": end,
    })

cov = dedup.groupby("symbol", sort=False).apply(coverage_per_symbol)

# See worst and best coverage
print("Worst coverage:\n", cov.sort_values("coverage_pct").head(10))
print("\nBest coverage:\n",  cov.sort_values("coverage_pct").tail(10))



C:\Users\alexp\AppData\Local\Temp\ipykernel_35164\442640303.py:21: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df["ts_min"] = df["ts_event"].dt.floor("T")
C:\Users\alexp\AppData\Local\Temp\ipykernel_35164\442640303.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full = pd.date_range(start=start, end=end, freq="T", tz="UTC")


Worst coverage:
             minutes_present  minutes_expected  minutes_missing  coverage_pct  \
symbol                                                                         
ESU6 C7650                2              8807             8805      0.000227   
ESH6 P5550                2              8806             8804      0.000227   
ESH6 C7850                2              8567             8565      0.000233   
ESZ6 P3200                2              8275             8273      0.000242   
ESU6 P6550                2              7746             7744      0.000258   
ESZ6 P5650                2              7320             7318      0.000273   
ESZ6 C8000                2              7244             7242      0.000276   
ESZ5 C8700                2              7236             7234      0.000276   
ESM6 P4400                2              7229             7227      0.000277   
ESU6 P3200                2              7148             7146      0.000280   

                      

C:\Users\alexp\AppData\Local\Temp\ipykernel_35164\442640303.py:43: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cov = dedup.groupby("symbol", sort=False).apply(coverage_per_symbol)


In [15]:
sym = "ESZ5 C6800"  # change as you like
g = dedup[dedup['symbol'] == sym].set_index('ts_min').sort_index()
rng = all_minutes[(all_minutes >= g.index.min()) & (all_minutes <= g.index.max())]
missing_minutes = rng.difference(g.index)
print(sym, "has", len(g), "bars and", len(missing_minutes), "missing minutes.")
missing_minutes[:20]


ESZ5 C6800 has 312 bars and 3729 missing minutes.


DatetimeIndex(['2025-10-15 00:15:00+00:00', '2025-10-15 00:32:00+00:00',
               '2025-10-15 00:39:00+00:00', '2025-10-15 00:43:00+00:00',
               '2025-10-15 00:59:00+00:00', '2025-10-15 01:03:00+00:00',
               '2025-10-15 01:09:00+00:00', '2025-10-15 01:12:00+00:00',
               '2025-10-15 01:43:00+00:00', '2025-10-15 01:47:00+00:00',
               '2025-10-15 01:48:00+00:00', '2025-10-15 01:51:00+00:00',
               '2025-10-15 01:58:00+00:00', '2025-10-15 01:59:00+00:00',
               '2025-10-15 02:03:00+00:00', '2025-10-15 02:11:00+00:00',
               '2025-10-15 02:16:00+00:00', '2025-10-15 02:22:00+00:00',
               '2025-10-15 02:44:00+00:00', '2025-10-15 02:46:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

In [16]:
dedup['date_utc'] = dedup['ts_min'].dt.date
day_counts = dedup.groupby(['symbol','date_utc']).size().unstack(fill_value=0)
day_counts.head()


date_utc,2025-10-15,2025-10-16,2025-10-17,2025-10-19,2025-10-20,2025-10-21
symbol,,,,,,
ESH6 C6000,0,2,0,0,0,0
ESH6 C6500,0,4,0,0,2,0
ESH6 C6550,0,4,3,0,0,0
ESH6 C6600,0,0,2,0,0,1
ESH6 C6650,5,0,1,0,2,1
